In [37]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [38]:
LINK_LIB_GR = ["torneo=1048", "torneo=1047", "torneo=1046", "torneo=1045", 
                "torneo=1044", "torneo=1043", "torneo=1074", "torneo=1089"]

LINK_LIB_IDA_VUELTA = ["torneo=1048", "torneo=1047", "torneo=1046", "torneo=1045", 
                        "torneo=1044"]

LINK_LIB_UNICA = ["torneo=1043", "torneo=1074", "torneo=1089", "libertadores"]

In [39]:
DICC_MES = {'Enero': '1', 'Febrero': '2', 'Marzo': '3', 'Abril': '4', 'Mayo': '5', 'Junio': '6', 'Julio': '7', 'Agosto': '8', 
            'Septiembre': '9', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'}

DICT_EQUIPOS = {}

In [40]:
def revisar_equipo(l, equipo):
    for i in l:
        eq = i.replace(",", "")
        eq = eq.replace(".", "")
        eq = eq.replace(" ", "")
        if eq[:3].upper() == equipo.upper():
            return i
        elif eq == "Dep.Cali" and equipo == "CAL":
            return i

In [41]:
def im_a_equipo(l):
    paises = []
    for i in l:
        im = i.find("img")
        im = im.get("src")
        im = im.split("/")[2].split(".")[0]
        paises.append(im)
    return paises

In [42]:
locales = []
visitas = []
gol_locales = []
gol_visitas = []
ganador = []
goles = []
resultados = []
dif_goles = []
dias = []
anos = []
fases = []

Primero definimos los partidos de grupo

In [43]:
ano = 2014
for num in LINK_LIB_GR:
    url = f"https://www.promiedos.com.ar/{num}"
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_grupos = sopa.find_all('div', class_ = 'grupo')
    for data in lista_grupos:
        equipos = data.find('table', id='posiciones')
        equipos = equipos.find_all('td', align = 'left')
        imagen = im_a_equipo(equipos)
        equipos = [i.text.replace(" ", "") for i in equipos]
        partidos = data.find_all('div', id = 'fixgrupo')

        for i in range(4):
            DICT_EQUIPOS[equipos[i]] = imagen[i]

        for datos in partidos:
            horario = datos.find_all('div', class_ = 'diahoragr')
            res = datos.find_all('div', class_ = 'grtr')

            for j in range(2):
                eq1 = res[j].find('div', class_ = 'greq1')
                eq1 = revisar_equipo(equipos, eq1.text)
                locales.append(eq1)
                eq2 = res[j].find('div', class_ = 'greq2')
                eq2 = revisar_equipo(equipos, eq2.text)
                visitas.append(eq2)

                resultado = res[j].find('div', class_ = 'grres4')
                resultado = resultado.text
                resultado = resultado.split('-')
                res1 = int(resultado[0])
                res2 = int(resultado[1])
                gol_locales.append(res1)
                gol_visitas.append(res2)
                resultados.append(str(res1) + "-" + str(res2))

                if res1 == res2:
                    ganador.append("Empate")
                    dif_goles.append(0)

                elif int(res1) > int(res2):
                    ganador.append("Local")
                    dif_goles.append(int(res1) - int(res2))

                elif int(res1) < int(res2):
                    ganador.append("Visita")
                    dif_goles.append(int(res2) - int(res1))

                goles.append(int(res1) + int(res2))
                anos.append(ano)
                dia = horario[j].text
                dia = dia.replace(",", "") ; dia = dia.split(" ")
                dia = dia[3]
                dias.append(dia)
                fases.append("Grupos")

    ano += 1


Añadimos las libertadores con finales ida y vuelta

In [44]:
LISTA_RONDAS = ["Final", "Final", "Semifinal", "Semifinal", "Cuartos", "Cuartos", "Octavos", "Octavos"]

In [45]:
for num in LINK_LIB_IDA_VUELTA:
    url = f"https://www.promiedos.com.ar/{num}"
    cont = 0
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_fases = sopa.find_all('div', id='partoff')
    for data in lista_fases:
        lista_partidos = data.find_all('div', id='fixturein')
        fase = LISTA_RONDAS[cont]
        for datos in lista_partidos:
            equ = datos.find_all('td', class_='game-t1')
            equ1 = equ[1].text ; equ2 = equ[2].text
            locales.append(equ1)
            visitas.append(equ2)

            res1 = datos.find_all('td', class_='game-r1')
            res1 = (res1[1].text)[0]
            gol_locales.append(int(res1))
            res2 = datos.find_all('td', class_='game-r2')
            res2 = (res2[1].text)[0]
            gol_visitas.append(int(res2))

            if int(res1) == int(res2):
                ganador.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                ganador.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                ganador.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))
            resultados.append(res1 + "-" + res2)


            dia = datos.find_all('tr', class_='diapart')
            dia = dia[0].text
            dia = dia.split(" ")
            anos.append(dia[4])
            ano = dia[4]
            dia = dia[1] + "/"  + DICC_MES[dia[3]]
            dias.append(dia)

            fases.append(fase)

        cont += 1

Añadimos las libertadores con final unica

In [46]:
for num in LINK_LIB_UNICA:
    url = f"https://www.promiedos.com.ar/{num}"
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    final_unica = sopa.find_all('div', id='partoff1')
    equ = final_unica[0].find_all('td', class_='game-t1')
    equ = final_unica[0].find_all('td', class_='game-t1')
    equ1 = equ[1].text.replace(' ', '') ; equ2 = equ[2].text.replace(' ', '')
    locales.append(equ1)
    visitas.append(equ2)
    res1 = final_unica[0].find_all('td', class_='game-r1')
    res1 = (res1[1].text)[0]
    gol_locales.append(int(res1))
    res2 = final_unica[0].find_all('td', class_='game-r2')
    res2 = (res2[1].text)[0]
    gol_visitas.append(int(res2))

    if int(res1) == int(res2):
        ganador.append("Empate")
        dif_goles.append(0)

    elif int(res1) > int(res2):
        ganador.append("Local")
        dif_goles.append(int(res1) - int(res2))

    elif int(res1) < int(res2):
        ganador.append("Visita")
        dif_goles.append(int(res2) - int(res1))

    goles.append(int(res1) + int(res2))
    resultados.append(res1 + "-" + res2)


    dia = final_unica[0].find_all('tr', class_='diapart')
    dia = dia[0].text
    dia = dia.split(" ")
    if len(dia) <= 4:
        dia.append("2022")
    anos.append(dia[4])
    ano = dia[4]
    dia = dia[1] + '/' + DICC_MES[dia[3]]
    dias.append(dia)
    fases.append("Final")

In [47]:
for num in LINK_LIB_UNICA:
    url = f"https://www.promiedos.com.ar/{num}"
    cont = 2
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_fases = sopa.find_all('div', id='partoff')
    for data in lista_fases:
        lista_partidos = data.find_all('div', id='fixturein')
        fase = LISTA_RONDAS[cont]
        for datos in lista_partidos:
            equ = datos.find_all('td', class_='game-t1')
            equ1 = equ[1].text ; equ2 = equ[2].text
            locales.append(equ1)
            visitas.append(equ2)

            res1 = datos.find_all('td', class_='game-r1')
            res1 = (res1[1].text)[0]
            gol_locales.append(int(res1))
            res2 = datos.find_all('td', class_='game-r2')
            res2 = (res2[1].text)[0]
            gol_visitas.append(int(res2))

            if int(res1) == int(res2):
                ganador.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                ganador.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                ganador.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))
            resultados.append(res1 + "-" + res2)


            dia = datos.find_all('tr', class_='diapart')
            dia = dia[0].text
            dia = dia.split(" ")
            if len(dia) < 5:
                anos.append("2022")
            else:
                anos.append(dia[4])
            dia = dia[1] + "/"  + DICC_MES[dia[3]]
            dias.append(dia)

            fases.append(fase)

        cont += 1
        if cont == 8:
            break

In [ ]:
url = "https://www.promiedos.com.ar/libertadores=faseprevia"

In [48]:
df_libertadores = pd.DataFrame()
df_libertadores["Local"] = locales
df_libertadores["Visita"] = visitas
df_libertadores["Gol_local"] = gol_locales
df_libertadores["Gol_visita"] = gol_visitas
df_libertadores["Ganador"] = ganador
df_libertadores["Resultado"] = resultados
df_libertadores["Año"] = anos
df_libertadores["Fase"] = fases
df_libertadores["Goles"] = goles
df_libertadores["Diferencia_goles"] = dif_goles
df_libertadores["Dia"] = dias
df_libertadores

,Local,Visita,Gol_local,Gol_visita,Ganador,Resultado,Año,Fase,Goles,Diferencia_goles,Dia
0,Universitario(P),Velez,0,1,Visita,0-1,2014,Grupos,1,1,11/2
1,Paranaense,TheStrongest,1,0,Local,1-0,2014,Grupos,1,1,13/2
2,TheStrongest,Universitario(P),1,0,Local,1-0,2014,Grupos,1,1,20/2
3,Velez,Paranaense,2,0,Local,2-0,2014,Grupos,2,2,25/2
4,TheStrongest,Velez,2,0,Local,2-0,2014,Grupos,2,2,11/3
...,...,...,...,...,...,...,...,...,...,...,...
1111,Emelec,Atl. Mineiro,1,1,Empate,1-1,2022,Octavos,2,0,28/6
1112,Dep. Tolima,Flamengo,0,1,Visita,0-1,2022,Octavos,1,1,29/6
1113,Corinthians,Boca Juniors,0,0,Empate,0-0,2022,Octavos,0,0,28/6
1114,Talleres (C),Colon,1,1,Empate,1-1,2022,Octavos,2,0,29/6


In [52]:
test = df_libertadores[df_libertadores["Fase"] == "Grupos"]
test = test[test["Año"] == "2022"]
test

,Local,Visita,Gol_local,Gol_visita,Ganador,Resultado,Año,Fase,Goles,Diferencia_goles,Dia
